In [ ]:
import requests
import pandas as pd
from weasyprint import HTML
from pathlib import Path
import os
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter

BASE_URL = "https://chilamboli.devmorphix.com"
ASSETS_DIR = "assets"
OUTPUT_DIR = "output"
OUTPUT_SUBDIR = "score sheet"
TEMPLATE_PATH = os.path.join(ASSETS_DIR, "scoring_sheet_template.html")
FONT_PATH = os.path.join(ASSETS_DIR, "Chilanka-Regular.ttf")

# Create output directories if they don't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, OUTPUT_SUBDIR), exist_ok=True)

print(f"Config loaded: BASE_URL={BASE_URL}")
print(f"Assets directory: {ASSETS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Template: {TEMPLATE_PATH}")

In [ ]:
def fetch_event_registrations(event_id):
    """Fetch event and its registrations from API (for scoring sheet before event; no results/judgments required)"""
    url = f"{BASE_URL}/api/admin/events/{event_id}/registrations"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    data = response.json()
    
    if not data.get("success"):
        raise RuntimeError(data.get("message", "API error"))
    
    return data

def load_template():
    """Load HTML template"""
    if not os.path.exists(TEMPLATE_PATH):
        raise FileNotFoundError(f"Template file not found: {TEMPLATE_PATH}")
    with open(TEMPLATE_PATH, 'r', encoding='utf-8') as f:
        return f.read()

print("Helper functions defined: fetch_event_registrations, load_template")

In [ ]:
def escape_html(text):
    """Escape HTML special characters"""
    if text is None:
        return ""
    text = str(text)
    return (text.replace('&', '&amp;')
                .replace('<', '&lt;')
                .replace('>', '&gt;')
                .replace('"', '&quot;')
                .replace("'", '&#x27;'))

def create_scoring_sheet_pdf(event_data, output_filename, folder_name=None):
    """Create a PDF scoring sheet from HTML template (portrait, for one judge)."""
    event = event_data["event"]
    registrations = event_data["registrations"]
    template = load_template()
    
    # Build table rows: chest number, 5 blank columns, total (blank), portal tick (blank)
    table_rows = ""
    for reg in registrations:
        chest = escape_html(reg.get("chestNumber") or "-")
        row = '<tr class="results-data-row">'
        row += f'<td class="col-chest">{chest}</td>'
        for _ in range(5):
            row += '<td class="col-mark"></td>'
        row += '<td class="col-total"></td>'
        row += '<td class="col-portal"></td>'
        row += '</tr>'
        table_rows += row
    
    html_content = template.replace('{{event_name}}', escape_html(event.get("name", "")))
    html_content = html_content.replace('{{category}}', escape_html(event.get("ageCategory", "")))
    html_content = html_content.replace('{{event_type}}', escape_html(event.get("eventType", "")))
    html_content = html_content.replace('{{table_rows}}', table_rows)
    
    folder = folder_name if folder_name is not None else output_filename.replace('.pdf', '')
    report_folder_path = os.path.join(OUTPUT_DIR, OUTPUT_SUBDIR, folder)
    os.makedirs(report_folder_path, exist_ok=True)
    pdf_path = os.path.join(report_folder_path, output_filename)
    html_path = os.path.join(report_folder_path, output_filename.replace('.pdf', '.html'))
    
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    HTML(string=html_content, base_url=Path(ASSETS_DIR).absolute()).write_pdf(pdf_path)
    print(f"✓ PDF generated: {pdf_path}")
    print(f"✓ HTML generated: {html_path}")


def _format_worksheet(worksheet, df, min_widths=None, max_widths=None):
    """Format worksheet with styled headers and auto-sized columns (matches other XL sheets)."""
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=11)

    for cell in worksheet[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)

    min_widths = min_widths or {}
    max_widths = max_widths or {}
    for idx, column in enumerate(df.columns, 1):
        column_letter = get_column_letter(idx)
        min_w = min_widths.get(str(column), 10)
        max_w = max_widths.get(str(column), 50)
        max_length = max(
            len(str(column)),
            df[column].astype(str).map(len).max() if len(df) > 0 else 0
        )
        worksheet.column_dimensions[column_letter].width = min(max(max_length + 2, min_w), max_w)

    worksheet.freeze_panes = "A2"
    worksheet.row_dimensions[1].height = 25


def create_registrations_excel(event_data, output_filename, folder_name=None):
    """Create an Excel sheet with all registrations for the event."""
    event = event_data["event"]
    registrations = event_data["registrations"]
    is_individual = event.get("eventType") == "Individual"
    name_col = "Student Name" if is_individual else "Team Name"

    folder = folder_name if folder_name is not None else output_filename.replace('.xlsx', '')
    report_folder_path = os.path.join(OUTPUT_DIR, OUTPUT_SUBDIR, folder)
    os.makedirs(report_folder_path, exist_ok=True)
    excel_path = os.path.join(report_folder_path, output_filename)

    df = pd.DataFrame([
        {
            "Chest Number": reg.get("chestNumber") or "-",
            name_col: reg.get("displayName") or reg.get("teamName") or "",
            "School Name": reg.get("schoolName") or "-",
        }
        for reg in registrations
    ])

    with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name="Registrations", index=False)
        _format_worksheet(writer.sheets["Registrations"], df, min_widths={"School Name": 55}, max_widths={"School Name": 65})

    print(f"✓ Excel generated: {excel_path}")


print("PDF and Excel generation functions defined: create_scoring_sheet_pdf, create_registrations_excel")

In [ ]:
# Generate scoring sheet for one event
EVENT_ID = input("Enter event ID: ").strip()
if not EVENT_ID:
    raise ValueError("Event ID is required. Please run the cell again and enter an event ID.")

print(f"Fetching registrations for event ID: {EVENT_ID}")
try:
    event_data = fetch_event_registrations(EVENT_ID)
    event_name = event_data["event"]["name"].replace(" ", "_").replace("/", "_")
    category = event_data["event"]["ageCategory"]
    folder_name = f"{category} - {event_name}"
    output_filename = f"{event_name}.pdf"
    excel_filename = f"{event_name}_registrations.xlsx"
    create_scoring_sheet_pdf(event_data, output_filename, folder_name=folder_name)
    create_registrations_excel(event_data, excel_filename, folder_name=folder_name)
    report_folder_path = os.path.join(OUTPUT_DIR, OUTPUT_SUBDIR, folder_name)
    print(f"\n✓ Scoring sheet and registrations Excel generated successfully!")
    print(f"  Event: {event_data['event']['name']}")
    print(f"  Category: {event_data['event']['ageCategory']}")
    print(f"  Rows: {len(event_data['registrations'])} registration(s)")
    print(f"  Output folder: {report_folder_path}")
    print(f"  PDF: {output_filename}")
    print(f"  Excel: {excel_filename}")
except Exception as e:
    print(f"Error generating scoring sheet: {e}")
    import traceback
    traceback.print_exc()